In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import combinations
from itertools import chain
from collections import namedtuple
import pickle
import tensorflow as tf
import tflearn
import matplotlib.pyplot as plt
import os.path
import shutil
import inspect
import matplotlib.pyplot as plt
import json
from pprint import pprint
from copy import copy
from tflearn.datasets import cifar10
from nn_lib import *
from train_utils import *
from train_procedures import *
np.random.seed(1997)
# %load_ext autoreload
# %autoreload 2
ALGORITHM=sys.argv[1]
BASE_DIR=sys.argv[2]
if ALGORITHM=='full_data':
    CLASS_BATCH_SIZE=int(sys.argv[3])

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
data_dict={}
(data_dict['X_train'],data_dict['Y_train']),(data_dict['X_test'],data_dict['Y_test'])=cifar10.load_data('./')
data_dict['Y_test']=np.array(data_dict['Y_test'])

In [3]:
# for testing purposes
ALGORITHM="SupportNet"

In [4]:
#non-incremental settings
if ALGORITHM == 'ft':
    hyper_params={'beta':1e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':0,
                  'se':True,
                  'primary_exemplars':None,
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':0,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar10',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':10,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True}
#icarl settings
elif ALGORITHM == 'icarl':
    hyper_params={'beta':1e-3,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':5,
                  'se':True,
                  'primary_exemplars':'icarl_exemplars',
                  'train_method':'train_distillation',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':0,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar10',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':10,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True}
#SupportNet settings
elif ALGORITHM == 'SupportNet':
    hyper_params={'beta':1e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':5,
                  'se':True,
                  'primary_exemplars':'svm_exemplars',
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':1e-2,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar10',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':10,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True}
#ewc settings
elif ALGORITHM == 'ewc':
    hyper_params={'beta':2e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':0,
                  'se':True,
                  'primary_exemplars':None,
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':1e-2,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar10',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':10,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True}
elif ALGORITHM == 'full_data':
    hyper_params={'beta':1e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[49,63],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':0,
                  'se':True,
                  'primary_exemplars':None,
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':0,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'cifar10',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':10,
                  'base_dir':BASE_DIR,
                  'class_batch_size':CLASS_BATCH_SIZE,
                  'use_theoretical_mean':True}
else:
    assert False

In [5]:
tf_tensors,tf_variables,tf_networks=build_graph(hyper_params,fixed_params)

Use ResNet32


/home/liz0f/SupportNet/train_procedures.py:50: UserWarning: when use ewc, loss_function should be softmax
  warnings.warn('when use ewc, loss_function should be softmax')


Use ResNet32
Use ResNet32


In [6]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
sess=tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
if ALGORITHM=='full_data':
    fit(tf_tensors,tf_variables,tf_networks,fixed_params,hyper_params,data_dict,sess,resume=False,
            save_params=True,evaluation_metric='top1_accuracy',save_history=True,
            num_epochs=70,num_iterations=1,verbose=2,print_freq=1,pretrain_evaluation=1,override_warning=False)
else:
    fit(tf_tensors,tf_variables,tf_networks,fixed_params,hyper_params,data_dict,sess,resume=False,
            save_params=True,evaluation_metric='top1_accuracy',save_history=True,
            num_epochs=70,num_iterations=5,verbose=2,print_freq=1,pretrain_evaluation=1,override_warning=False)

{'base_dir': './Aug25-cifar10-icarl-debug2',
 'class_batch_size': 2,
 'dataset': 'cifar10',
 'net_type': 'ResNet32',
 'random_seed': 1997,
 'total_num_classes': 10,
 'use_theoretical_mean': True}
{'beta': 0.0002,
 'ewc_lambda': 0,
 'exemplars_set_size': 2000,
 'final_train_epochs': 5,
 'initial_lr': 0.1,
 'loss_function': 'sigmoid_cross_entropy_with_logits',
 'lr_reduction_epoch': [49, 63],
 'lr_reduction_rate': 10,
 'optimizer': 'momentum',
 'primary_exemplars': 'icarl_exemplars',
 'sample_weight': 0,
 'se': True,
 'shuffle_class_ord': False,
 'test_batch_size': 64,
 'train_batch_size': 64,
 'train_method': 'train_distillation',
 'use_fixedsize_exemplars': True}
./Aug25-cifar10-icarl-debug2 already exists, override?y
Y_train class info:
Counter({1: 5000, 0: 5000})
Y_test class info:
Counter({0: 1000, 1: 1000})
===========Iteration 1=============
Using classes [0, 1]


/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Class batch pretrain evaluation (plain method)
	Train loss: 0.361058
	Train class loss: 0.069459
	Train reg loss: 0.291599
	Validation loss: 0.361188
	Validation class loss: 0.069589
	Validation reg loss: 0.291599
	Top1 train accuracy: 0.392800
	Top5 train accuracy: 1.000000
	Top1 validation accuracy: 0.367500
	Top5 validation accuracy: 1.000000
[[118 865  17]
 [371 617  12]
 [  0   0   0]]
Epoch 1
	Train loss: 0.375489
	Train class loss: 0.090525
	Train reg loss: 0.284964
	Validation loss: 0.282078
	Validation class loss: 0.004748
	Validation reg loss: 0.277330
	Top1 train accuracy: 0.810600
	Top5 train accuracy: 0.997800
	Top1 validation accuracy: 0.904000
	Top5 validation accuracy: 1.000000
[[876 124]
 [ 68 932]]
mbi=275436032
time= 29.415866
saving model parameters...
Epoch 2
	Train loss: 0.321500
	Train class loss: 0.052528
	Train reg loss: 0.268971
	Validation loss: 0.266524
	Validation class loss: 0.004974
	Validation reg loss: 0.261550
	Top1 train accuracy: 0.891700
	Top5 train

Epoch 22
	Train loss: 0.097737
	Train class loss: 0.010740
	Train reg loss: 0.086996
	Validation loss: 0.086250
	Validation class loss: 0.001396
	Validation reg loss: 0.084855
	Top1 train accuracy: 0.978500
	Top5 train accuracy: 0.998400
	Top1 validation accuracy: 0.976500
	Top5 validation accuracy: 1.000000
[[990  10]
 [ 37 963]]
mbi=275436032
time= 26.150210
Epoch 23
	Train loss: 0.093870
	Train class loss: 0.011307
	Train reg loss: 0.082563
	Validation loss: 0.081789
	Validation class loss: 0.001228
	Validation reg loss: 0.080562
	Top1 train accuracy: 0.978500
	Top5 train accuracy: 0.998400
	Top1 validation accuracy: 0.980500
	Top5 validation accuracy: 1.000000
[[982  18]
 [ 21 979]]
mbi=275436032
time= 26.089044
Epoch 24
	Train loss: 0.089128
	Train class loss: 0.010694
	Train reg loss: 0.078433
	Validation loss: 0.077970
	Validation class loss: 0.001434
	Validation reg loss: 0.076536
	Top1 train accuracy: 0.978800
	Top5 train accuracy: 0.998400
	Top1 validation accuracy: 0.977500


Epoch 45
	Train loss: 0.040030
	Train class loss: 0.008278
	Train reg loss: 0.031752
	Validation loss: 0.032842
	Validation class loss: 0.001545
	Validation reg loss: 0.031298
	Top1 train accuracy: 0.984100
	Top5 train accuracy: 0.998400
	Top1 validation accuracy: 0.975000
	Top5 validation accuracy: 1.000000
[[956  44]
 [  6 994]]
mbi=275436032
time= 26.577914
Epoch 46
	Train loss: 0.040777
	Train class loss: 0.009960
	Train reg loss: 0.030817
	Validation loss: 0.031833
	Validation class loss: 0.001373
	Validation reg loss: 0.030460
	Top1 train accuracy: 0.979800
	Top5 train accuracy: 0.998400
	Top1 validation accuracy: 0.975500
	Top5 validation accuracy: 1.000000
[[991   9]
 [ 40 960]]
mbi=275436032
time= 26.556336
Epoch 47
	Train loss: 0.038119
	Train class loss: 0.008154
	Train reg loss: 0.029965
	Validation loss: 0.030548
	Validation class loss: 0.001010
	Validation reg loss: 0.029539
	Top1 train accuracy: 0.984800
	Top5 train accuracy: 0.998400
	Top1 validation accuracy: 0.984000


...

Epoch 24
	Train loss: 0.239999
	Train class loss: 0.206575
	Train reg loss: 0.033425
	Validation loss: 0.066931
	Validation class loss: 0.033404
	Validation reg loss: 0.033527
	Top1 train accuracy: 0.813917
	Top5 train accuracy: 0.989000
	Top1 validation accuracy: 0.929000
	Top5 validation accuracy: 0.983500
[[  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [ 50  25   0   1   4 880  40]
 [  9   9   1   0   0   3 978]]
mbi=275436032
time= 36.417164
saving model parameters...
Epoch 25
	Train loss: 0.238302
	Train class loss: 0.204868
	Train reg loss: 0.033434
	Validation loss: 0.073710
	Validation class loss: 0.040228
	Validation reg loss: 0.033482
	Top1 train accuracy: 0.821417
	Top5 train accuracy: 0.991750
	Top1 validation accuracy: 0.738000
	Top5 validation accuracy: 0.972000
[[  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]


Epoch 37
	Train loss: 0.238822
	Train class loss: 0.205360
	Train reg loss: 0.033462
	Validation loss: 0.070610
	Validation class loss: 0.037060
	Validation reg loss: 0.033550
	Top1 train accuracy: 0.821917
	Top5 train accuracy: 0.989083
	Top1 validation accuracy: 0.899000
	Top5 validation accuracy: 0.987000
[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [ 45   4   1   1   0   1   1   0 946   1]
 [ 65  45   1   4   2   1   2   1  27 852]]
mbi=275436032
time= 36.497685
Epoch 38
	Train loss: 0.238281
	Train class loss: 0.204851
	Train reg loss: 0.033430
	Validation loss: 0.073539
	Validation class loss: 0.040009
	Validation reg loss: 0.033529
	Top1 train accuracy: 0.828583
	Top5 train accuracy: 

Epoch 49
	Train loss: 0.237947
	Train class loss: 0.204553
	Train reg loss: 0.033394
	Validation loss: 0.075545
	Validation class loss: 0.041986
	Validation reg loss: 0.033559
	Top1 train accuracy: 0.832500
	Top5 train accuracy: 0.991500
	Top1 validation accuracy: 0.585500
	Top5 validation accuracy: 0.925500
[[  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [232  32   5   0   1   3   1 724   2]
 [170 371   0   1   3   2   0   6 447]]
mbi=275436032
time= 36.541533
lr reduced to 0.010000
Epoch 50
	Train loss: 0.227534
	Train class loss: 0.194115
	Train reg loss: 0.033419
	Validation loss: 0.069090
	Validation class loss: 0.035634
	Validation reg loss: 0.033456
	Top1 train accuracy: 0.849333
	Top5 train accuracy: 0.993750
	Top1 validation accuracy: 0.899000
	Top5 valid

Epoch 60
	Train loss: 0.215177
	Train class loss: 0.182988
	Train reg loss: 0.032189
	Validation loss: 0.067790
	Validation class loss: 0.035573
	Validation reg loss: 0.032217
	Top1 train accuracy: 0.891417
	Top5 train accuracy: 0.996250
	Top1 validation accuracy: 0.906000
	Top5 validation accuracy: 0.992000
[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [ 65  15   1   0   1   1   2   3 908   4]
 [ 30  50   1   3   1   1   1   3   6 904]]
mbi=275436032
time= 36.480444
Epoch 61
	Train loss: 0.214920
	Train class loss: 0.182857
	Train reg loss: 0.032063
	Validation loss: 0.067934
	Validation class loss: 0.035851
	Validation reg loss: 0.032083
	Top1 train accuracy: 0.892583
	Top5 train accuracy: 

On iteration 5
Plain evaluation before retrain
	Best top1 validation accuracy: 0.905500
	Best top5 validation accuracy: 0.993000
	Best top1 cumul accuracy: 0.703800
	Best top5 cumul accuracy: 0.984900
	Best top1 ori accuracy: 0.920500
	Best top5 ori accuracy: 0.999000
	Report string of cumul dataset
             precision    recall  f1-score   support

          0       0.33      0.91      0.49      1000
          1       0.89      0.93      0.91      1000
          2       0.81      0.33      0.47      1000
          3       0.62      0.56      0.59      1000
          4       0.79      0.56      0.65      1000
          5       0.81      0.58      0.68      1000
          6       0.92      0.74      0.82      1000
          7       0.94      0.62      0.75      1000
          8       0.90      0.90      0.90      1000
          9       0.89      0.91      0.90      1000

avg / total       0.79      0.70      0.72     10000

1608 support_vectors for class 0
709 support_vectors for cla

In [16]:
data_dict_iterations=data_dict_each_iteration("Aug31-cifar10-SupportNet-full2",5,2,data_dict)

0
[0, 1]
1
[2, 3]
2
[4, 5]
3
[6, 7]
4
[8, 9]


In [17]:
eval_vals_lists_test=eval_vals_each_iteration("Aug31-cifar10-SupportNet-full2",5,2,tf_tensors,tf_networks,data_dict_iterations,"cifar10",sess)

0


/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/liz0

1
2
3
4


In [18]:
for i in range(0,len(eval_vals_lists_test)):
    for j in range(0,len(eval_vals_lists_test)):
        print(eval_vals_lists_test[i][j]["top1_accuracy"],end=",")
    print()

0.9885,0.0,0.0,0.0,0.0,
0.898,0.9105,0.0,0.0,0.0,
0.8895,0.4795,0.932,0.0,0.0,
0.91,0.551,0.639,0.9655,0.0,
0.6395,0.5745,0.697,0.8205,0.972,
